In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Dropout, Dense, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam, Adadelta
import numpy as np
import tensorflow as tf
import keras

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
image_size=28
x_train=np.array(x_train).reshape(-1, image_size ,image_size ,1)
x_test=np.array(x_test).reshape(-1, image_size ,image_size ,1)
#scale image =1 channel and all the data are scale
print(x_train.shape)
print(x_test.shape)
x_train,x_test=x_train/255.0 , x_test/255.0
print(x_train.shape[1:])

(60000, 28, 28)
(60000, 28, 28, 1)
(10000, 28, 28, 1)
(28, 28, 1)


In [3]:
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape= x_train.shape[1:], kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

# 3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(10, activation='softmax', name='output'))

#Model Summary
AlexNet.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 7, 7, 96)          11712     
                                                                 
 activation (Activation)     (None, 7, 7, 96)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 4, 96)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 256)         614656    
                                                                 
 activation_1 (Activation)   (None, 4, 4, 256)         0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 2, 2, 256)        0         
 2D)                                                    

In [4]:
optimize = SGD(learning_rate=0.05)
AlexNet.compile(optimizer=optimize, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
AlexNet.fit(x_train, y_train, epochs=5 ,batch_size=20)
loss, accuracy= AlexNet.evaluate(x_test, y_test)
print( 'the Accuracy =' , accuracy*100)

Epoch 1/5
3000/3000 [==============================] - 31s 9ms/step - loss: 1.0805 - accuracy: 0.6014
Epoch 2/5
3000/3000 [==============================] - 26s 9ms/step - loss: 0.1048 - accuracy: 0.9707
Epoch 3/5
3000/3000 [==============================] - 26s 9ms/step - loss: 0.0642 - accuracy: 0.9819
Epoch 4/5
3000/3000 [==============================] - 26s 9ms/step - loss: 0.0460 - accuracy: 0.9866
Epoch 5/5
313/313 [==============================] - 2s 5ms/step - loss: 0.0464 - accuracy: 0.9870
the Accuracy = 98.69999885559082


In [21]:
y_pred=AlexNet.predict(x_test)
y_pre=np.argmax(y_pred,axis=1)
y_pred

313/313 [==============================] - 1s 3ms/step


array([[4.97869017e-08, 1.33139679e-06, 1.47998105e-06, ...,
        9.99959588e-01, 2.45883616e-08, 3.23991662e-05],
       [2.21421548e-09, 9.69254052e-11, 9.99999881e-01, ...,
        4.95085075e-08, 8.79287032e-10, 5.85815728e-12],
       [3.38461120e-07, 9.99891758e-01, 7.32776789e-06, ...,
        4.26362494e-05, 4.33862187e-05, 2.39385281e-06],
       ...,
       [2.36559716e-10, 3.57704664e-11, 7.47885487e-09, ...,
        1.35426115e-08, 4.73532458e-09, 6.04342347e-07],
       [1.42767269e-04, 7.38445806e-07, 3.47024724e-07, ...,
        1.10747493e-07, 4.61275689e-04, 1.74721223e-04],
       [6.60951969e-08, 5.24284505e-10, 4.49987922e-08, ...,
        2.03230466e-12, 2.45577052e-08, 1.06634736e-11]], dtype=float32)

In [19]:
from sklearn.metrics import f1_score,confusion_matrix,recall_score,precision_score
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pre))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.99      0.98      1032
           3       0.99      0.98      0.99      1010
           4       0.99      1.00      0.99       982
           5       0.99      0.99      0.99       892
           6       0.99      0.99      0.99       958
           7       0.98      0.98      0.98      1028
           8       0.98      0.98      0.98       974
           9       0.99      0.98      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [20]:
print(confusion_matrix(y_test,y_pre))

[[ 978    0    0    0    0    0    1    1    0    0]
 [   7 1125    1    0    0    0    1    0    1    0]
 [   4    1 1017    1    2    0    0    4    3    0]
 [   0    0    8  992    0    4    0    0    5    1]
 [   0    0    0    0  979    0    0    1    0    2]
 [   3    0    0    4    0  879    5    0    1    0]
 [   4    1    1    0    5    1  946    0    0    0]
 [   1    4    7    0    0    0    0 1011    2    3]
 [   3    1    3    1    1    1    3    2  957    2]
 [   0    3    0    1    5    1    0    8    5  986]]


In [5]:
model = Sequential()
model.add(Conv2D(input_shape= x_train.shape[1:],filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2), padding="same",strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2), padding="same",strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2), padding="same",strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2), padding="same",strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=10, activation="softmax"))

In [6]:
optimize = SGD(learning_rate=0.05)
model.compile(optimizer=optimize, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5 ,batch_size=20)
loss, accuracy= model.evaluate(x_test, y_test)
print( 'the Accuracy =' , accuracy*100)
     

Epoch 1/5
3000/3000 [==============================] - 78s 26ms/step - loss: 2.2986 - accuracy: 0.1180
Epoch 2/5
3000/3000 [==============================] - 77s 26ms/step - loss: 1.8752 - accuracy: 0.2901
Epoch 3/5
3000/3000 [==============================] - 77s 26ms/step - loss: 0.1167 - accuracy: 0.9669
Epoch 4/5
3000/3000 [==============================] - 77s 26ms/step - loss: 0.0543 - accuracy: 0.9849
Epoch 5/5
313/313 [==============================] - 4s 10ms/step - loss: 0.0278 - accuracy: 0.9906
the Accuracy = 99.05999898910522
